<a href="https://colab.research.google.com/github/ncsu-landscape-dynamics/GEE_Daymet_temp_and_RH_via_Python/blob/main/GEE_Daymet_RH_temp_via_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
!pip install geetools

import geetools
import folium
import ee

ee.Authenticate()


In [3]:
ee.Initialize()

# Some prelims:
# Outline of lower 48 U.S.
usbound = ee.Image("users/japolo/us_bounds")
usb = usbound.geometry()

#oeel = require('users/OEEL/lib:loadAll')
def clip_img(img):
    return ee.Image(img).clip(usb)


In [4]:
# Set up Daymet collection from 2009 to 2020
daymetb = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate('2009-05-29', '2009-05-31').map(clip_img)


In [5]:
# Set up projection for reproject of Daymet
proj = ee.Projection('EPSG:4326')

# Wrong
def imreproj(image):
    return image.reproject(crs='EPSG:4326', scale = 1000)

# Reproject Daymet
daymet = daymetb.map(imreproj)

# Vapor pressure from Daymet
dayvp = daymet.select('vp')

# Daylength
daydl = daymet.select('dayl')

# Set a threshold of temps for use in a later calculation
def threshold(image):
    return image.where(image.lt(-5.0), -5)

# Set up tmax and tmin
daytm = daymet.select('tmax','tmin')
# Apply threshold
daytm2 = daytm.map(threshold)

# Use an equals filter to define how the collections match.
filter = ee.Filter.equals(leftField = 'system:index', rightField = 'system:index')

# Create the join.
simpleJoin = ee.Join.simple()

# Applt join
mod1join = ee.ImageCollection(simpleJoin.apply(daytm2, daydl, filter))
mod2join = ee.ImageCollection(simpleJoin.apply(daydl, daytm2, filter))

def enhance(image):
    # Create a collection with 1 image
    temp = ee.ImageCollection(ee.List([image]))
    # Apply join to collection 2
    # Resulting collection will have 1 image with exact same date as img
    join = simpleJoin.apply(mod2join, temp, filter)
    # Get resulting image
    i2 = ee.Image(join.first())
    return image.addBands(i2)

step1 = mod1join.map(enhance)

TC = 4.0
P = 1.5

def temB4sunris(image):
  day_len = image.select('dayl').divide(3600)
  up_time = day_len.multiply(-0.5).add(12)
  set_time = day_len.multiply(0.5).add(12)
  nitelen = image.expression(
  'daylen * -1 + 24', {
    'daylen' : day_len
  })
  snsetT = image.expression(
  '(tmin + (tmax - tmin) * sin(pi * (daylen / (daylen + 2 * P))))', {
    'tmin' : image.select('tmin'),
    'tmax' : image.select('tmax'),
    'daylen' : day_len,
    'pi' : 3.14159,
    'P' : 1.5
    })
  snsetT2 = image.expression(
  '(tmin - snsetT * exp(-1 * nitelen / TC) + (snsetT - tmin) * exp(-1 * (hr + 24 - sunset) / TC)) / (1 - exp(-1 * nitelen / TC))', {
    'tmin' : image.select('tmin'),
    'snsetT' : snsetT,
    'nitelen' : nitelen,
    'tmax' : image.select('tmax'),
    'hr' : 0,
    'sunset' : set_time,
    'daylen' : day_len,
    'pi' : 3.14159,
    'P' : 1.5,
    'TC' : 4.0
    })
  snsetT2 = snsetT2.rename('T_early')
  return snsetT2.copyProperties(image, ['system:time_start']).copyProperties(image,['system:index'])


In [6]:
mnitetem09 = step1.map(temB4sunris)

mod1join = ee.ImageCollection(simpleJoin.apply(dayvp, mnitetem09, filter))
mod2join = ee.ImageCollection(simpleJoin.apply(mnitetem09, dayvp, filter))

tearlyvp = mod1join.map(enhance)


In [ ]:
# Morning temp climb
def tempclimb(image):
   day_len = image.select('dayl').divide(3600)
   up_time = day_len.multiply(-0.5).add(12)
   tempup = image.expression(
   'tmin + (tmax - tmin) * sin(pi * (hr - sunris) / (daylen + 2 * P))', {
    'tmin' : image.select('tmin'),
    'tmax' : image.select('tmax'),
    'pi' : 3.14159,
    'hr' : 0,
    'sunris' : up_time,
    'daylen' : day_len,
    'P' : 1.5
    })
   return tempup


In [ ]:
# After daily high T, before sunset
def tempdown(image):
   day_len = image.select('dayl').divide(3600)
   up_time = day_len.multiply(-0.5).add(12)
   tempdrop = ee.Image.expression(
    'tmin + (tmax - tmin) * sin(pi * (hr - sunris) / (daylen + 2 * P))', {
      'tmin' : image.select('tmin'),
      'tmax' : image.select('tmax'),
      'pi' : 3.14159,
      'hr' : 0,
      'sunris' : up_time,
      'daylen' : day_len,
      'P' : 1.5
   })
   return tempdown


In [ ]:
# before midnight
def tempnite = function(image) {
  day_len = image.select('dayl').divide(3600)
  up_time = day_len.multiply(-0.5).add(12)
  set_time = day_len.multiply(0.5).add(12)
  nitelen = ee.Image().expression(
  'daylen * -1 + 24', {
    'daylen' : day_len
  })
  snsetT = ee.Image().expression(
  '(tmin + (tmax - tmin) * sin(pi * (daylen / (daylen + 2 * P))))', {
    'tmin' : image.select('tmin'),
    'tmax' : image.select('tmax'),
    'daylen' : day_len,
    'pi' : 3.14159,
    'P' : 1.5
    })
  tempdark = ee.Image.expression(
  '(tmin - snsetT * exp(-1 * nitelen / TC) + (snsetT - tmin) * exp(-1 * (hr - sunset) / TC)) / (1 - exp(-1 * nitelen / TC))', {
    'tmin' : image.select('tmin'),
    'snsetT' : snsetT,
    'nitelen' : nitelen,
    'tmax' : image.select('tmax'),
    'hr' : i,
    'sunset' : set_time,
    'daylen' : day_len,
    'pi' : 3.14159,
    'P' : 1.5,
    'TC' : 4.0
    })
   return tempdark


In [7]:
def relhum(image):
    svp1 = ee.Image().expression(
    '611 * 10**(7.5 * temp / (237.7 + temp))',{  #pascals. use 6.11 for kpa
      'temp' : image.select('T_early')
    })
    svp2 = ee.Image().expression(
    'vp * 100 / svp1', {
      'vp' : image.select('vp'),
      'svp1' : svp1
    })
    svp2 = svp2.where(svp2.gt(100), 100)
    rel_h = svp2.where(svp2.lt(0), 0)
    rel_h = rel_h.rename('rel_hum')
    return ee.Image(rel_h).copyProperties(image, ['system:time_start']).copyProperties(image,['system:index'])


In [8]:
mnittmrh09 = tearlyvp.map(relhum)

mod1join = ee.ImageCollection(simpleJoin.apply(mnittmrh09, tearlyvp, filter))
mod2join = ee.ImageCollection(simpleJoin.apply(tearlyvp, mnittmrh09, filter))

tearlyvp = mod1join.map(enhance)


In [9]:

print(tearlyvp.first().getInfo())

{'type': 'Image', 'bands': [{'id': 'rel_hum', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [0.008983152841195215, 0, 0, 0, -0.008983152841195215, 0]}, {'id': 'vp', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [0.008983152841195215, 0, 0, 0, -0.008983152841195215, 0]}, {'id': 'T_early', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [0.008983152841195215, 0, 0, 0, -0.008983152841195215, 0]}], 'properties': {'system:time_start': 1243555200000, 'system:index': '20090529'}}


In [30]:
t_rh_2009_00 = tearlyvp.select(['T_early','rel_hum'])

In [31]:
scale = 1000
name = 't_rh_2009_00'
name_pattern = name+'_{system_date}'
## the keywords between curly brackets can be {system_date} for the date of the
## image (formatted using `date_pattern` arg), {id} for the id of the image
## and/or any image property. You can also pass extra keywords using the `extra`
## argument. Also, numeric values can be formatted using a format string (as
## shown in {WRS_PATH:%d} (%d means it will be converted to integer)
date_pattern = 'yMMMdd' # dd: day, MMM: month (JAN), y: year
folder = 'daymets_temp_hr'
data_type = 'uint8'
region1 = usb

# ## Export
tasks = geetools.batch.Export.imagecollection.toDrive(
            collection=t_rh_2009_00,
            folder=folder,
            namePattern=name_pattern,
            #scale=scale,
            dataType=data_type,
            datePattern=date_pattern,
            verbose=True,
            region=region1,
            maxPixels=int(1e13)
        )


EEException: ignored

In [11]:
# Not needed. Just for eyeballing some values in the images.
lat, lon = 37, -100

my_map = folium.Map(location=[lat, lon], zoom_start=4)
my_map

def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

lc_img1 = tearlyvp.select('T_early').first()
lc_img2 = tearlyvp.select('rel_hum').first()

lc_vis_params = {
    'min': 1,'max': 17,
    'palette': ['05450a','086a10', '54a708', '78d203', '009900', 'c6b044',
                'dcd159', 'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44',
                'a5a5a5', 'ff6d4c', '69fff8', 'f9ffa4', '1c0dff']
}

lc_vis_params2 = {
    'min': 1,'max': 99,
    'palette': ['05450a','086a10', '54a708', '78d203', '009900', 'c6b044',
                'dcd159', 'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44',
                'a5a5a5', 'ff6d4c', '69fff8', 'f9ffa4', '1c0dff']
}

my_map.add_ee_layer(lc_img1, lc_vis_params, 't')
my_map.add_ee_layer(lc_img2, lc_vis_params2, 'rh')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

